In [6]:
import numpy as np
import pandas as pd
import glob
import os
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
# from sklearn.grid_search import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict

In [7]:
dataFrame = pd.read_csv("./CombinedData.csv", header = 0)
dataFrame.insert(0, 'ID', range(0,len(dataFrame)))
print(dataFrame.head())

   ID         SOLD DATE PROPERTY TYPE                  ADDRESS        CITY  \
0   0      July-28-2022   Condo/Co-op      3881 Carl Parmer Dr  Harrisburg   
1   1    August-17-2022   Condo/Co-op        6419 Town Hall Pl  Harrisburg   
2   2  February-10-2022   Condo/Co-op  1 Buffalo Ave NW Unit 7     Concord   
3   3      March-4-2022   Condo/Co-op          476 Camrose Cir     Concord   
4   4    October-8-2021   Condo/Co-op      4068 Town Center Rd  Harrisburg   

  STATE OR PROVINCE ZIP OR POSTAL CODE   PRICE  BEDS  BATHS  \
0                NC              28075  258000   3.0    2.0   
1                NC              28075  242000   3.0    2.0   
2                NC              28025  102000   2.0    1.0   
3                NC              28025  203000   2.0    1.5   
4                NC              28075  170000   2.0    1.0   

                 LOCATION  SQUARE FEET  LOT SIZE  YEAR BUILT  $/SQUARE FEET  \
0              Harrisburg       1181.0       NaN      2005.0          218

In [8]:
dataFrame = dataFrame[dataFrame['SQUARE FEET'].notna()]
dataFrame = dataFrame[dataFrame['SOLD DATE'].notna()]
dataFrame = dataFrame[dataFrame['ZIP OR POSTAL CODE'].notna()]

In [9]:
dataFrame['SOLD DATE'] = pd.to_datetime(dataFrame['SOLD DATE']).apply(lambda x: x.value)

In [17]:
dataFrame['ZIP OR POSTAL CODE'] = pd.Series([x.split('-')[0] if '-' in x else x for x in dataFrame['ZIP OR POSTAL CODE']])

In [17]:
dataFrame['SQUARE FEET']

0        1181.0
1        1186.0
2         819.0
3        1188.0
4         877.0
          ...  
78961    1055.0
78965    1800.0
78996    1552.0
79020    2086.0
79023    1555.0
Name: SQUARE FEET, Length: 43872, dtype: float64

In [13]:
dataFrame['SOLD DATE']

0        1658966400000000000
1        1660694400000000000
2        1644451200000000000
3        1646352000000000000
4        1633651200000000000
                ...         
78961    1641340800000000000
78965    1646784000000000000
78996    1637712000000000000
79020    1637020800000000000
79023    1637625600000000000
Name: SOLD DATE, Length: 43872, dtype: int64

In [15]:
dataFrame = dataFrame.drop(columns=["URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)", 
                                    "LOCATION"])

In [16]:
train_dataset, test_dataset = train_test_split(dataFrame, test_size=.3)
print(train_dataset.head())

          ID            SOLD DATE              PROPERTY TYPE  \
76339  76339  1632268800000000000                  Townhouse   
63064  63064  1634860800000000000  Single Family Residential   
63990  63990  1641859200000000000  Single Family Residential   
50587  50587  1632096000000000000  Single Family Residential   
39283  39283  1630540800000000000  Single Family Residential   

                               ADDRESS          CITY STATE OR PROVINCE  \
76339  10422 Glenmere Creek Cir Lot 17     Charlotte                NC   
63064                6007 Graystone Dr   Chapel Hill                NC   
63990                   2409 Summit Dr  Hillsborough                NC   
50587                204 Lake Shore Dr        Benson                NC   
39283                   203 Cottage Ln        Durham                NC   

      ZIP OR POSTAL CODE    PRICE  BEDS  BATHS  SQUARE FEET  LOT SIZE  \
76339              28262   382500   3.0    2.5       2262.0    1742.0   
63064              27514

In [17]:
category_features = ['PROPERTY TYPE', 'CITY', 'ADDRESS', 'STATE OR PROVINCE', 'ZIP OR POSTAL CODE']
every_column_except_y = [col for col in train_dataset.columns if col not in [ 'PRICE', 'ID']]
train_dataset.describe()

,ID,SOLD DATE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,30710.000000,3.071000e+04,3.071000e+04,30691.000000,30695.000000,30710.000000,2.893900e+04,30692.000000,30710.000000,15882.000000,30710.000000,30710.000000
mean,39149.569000,1.646048e+18,4.257931e+05,3.265192,2.448982,2074.794855,5.120268e+04,1994.250586,210.687952,104.495655,35.600316,-80.205816
std,21944.382113,9.139573e+15,2.942860e+05,0.930855,0.878706,970.366583,2.244734e+06,28.259590,473.735267,105.963528,0.301232,1.251453
min,0.000000,1.630454e+18,2.450000e+02,0.000000,0.500000,2.000000,1.000000e+01,1647.000000,0.000000,1.000000,35.016758,-82.867112
25%,20979.750000,1.638144e+18,2.789900e+05,3.000000,2.000000,1408.000000,7.405000e+03,1979.000000,161.000000,30.000000,35.352807,-80.895258
50%,40829.500000,1.646266e+18,3.670000e+05,3.000000,2.500000,1856.000000,1.306800e+04,2003.000000,193.000000,65.000000,35.579651,-80.625299
75%,56308.500000,1.653955e+18,4.840990e+05,4.000000,3.000000,2527.000000,2.744200e+04,2018.000000,233.000000,155.000000,35.869880,-78.940903
max,78996.000000,1.661990e+18,6.800000e+06,32.000000,32.000000,17988.000000,3.775781e+08,2023.000000,62500.000000,3500.000000,36.538558,-78.086563


In [18]:
non_categorical_columns = [col for col in train_dataset.columns if col not in category_features and col not in ['ID']]
numeric_features = train_dataset[non_categorical_columns].dtypes[train_dataset.dtypes != "object"].index
train_dataset[numeric_features] = np.log1p(train_dataset[numeric_features])

non_categorical_columns = [col for col in test_dataset.columns if col not in category_features and col not in ['ID']]
numeric_features = test_dataset[non_categorical_columns].dtypes[test_dataset.dtypes != "object"].index
test_dataset[numeric_features] = np.log1p(test_dataset[numeric_features])

c:\python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
# nan_features = ['LOCATION']
# def ConvertToNAString(data, columnsList):
#     for x in columnsList:
#         data[x] = str(data[x])
# ConvertToNAString(train_dataset, nan_features)
# ConvertToNAString(test_dataset, nan_features)

In [20]:
def createColumnPerValue(data, columnsList):
  for x in columnsList:
    values = pd.unique(data[x])

    for y in values: 
      column_name = x + "_" + str(y)
      data[column_name]=(data[x]==y).astype(float)
    
    data.drop(x, axis=1, inplace=True)
    
train_dataset = pd.get_dummies(train_dataset,columns =category_features)
test_dataset = pd.get_dummies(test_dataset,columns =category_features)

In [21]:
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [22]:
train_dataset['SOLD DATE']

76339    41.936503
63064    41.938089
63990    41.942361
50587    41.936397
39283    41.935443
           ...    
75545    41.937349
23169    41.948656
40619    41.946143
54801    41.935390
11329    41.938459
Name: SOLD DATE, Length: 30710, dtype: float64

In [23]:
#df['time'] = df['time'].apply(lambda x: x.value)

In [24]:
train_dataset.columns

Index(['ID', 'SOLD DATE', 'PRICE', 'BEDS', 'BATHS', 'SQUARE FEET', 'LOT SIZE',
       'YEAR BUILT', '$/SQUARE FEET', 'HOA/MONTH',
       ...
       'ZIP OR POSTAL CODE_28803', 'ZIP OR POSTAL CODE_28804',
       'ZIP OR POSTAL CODE_28805', 'ZIP OR POSTAL CODE_28806',
       'ZIP OR POSTAL CODE_28816', 'ZIP OR POSTAL CODE_29269',
       'ZIP OR POSTAL CODE_29482', 'ZIP OR POSTAL CODE_29710',
       'ZIP OR POSTAL CODE_57510', 'ZIP OR POSTAL CODE_98078'],
      dtype='object', length=32188)

In [25]:
train_dataset['PRICE']

76339    12.854487
63064    13.995747
63990    12.923915
50587    12.911645
39283    12.644331
           ...    
75545    12.259618
23169    13.270785
40619    12.742569
54801    12.409018
11329    12.128117
Name: PRICE, Length: 30710, dtype: float64

In [ ]:
every_column_except_y= [col for col in train_dataset.columns if col not in ['PRICE','ID']]
model.fit(train_dataset[every_column_except_y].head(5),train_dataset['PRICE'].head(5))
OrderedDict(sorted(model.booster().get_fscore().items(), key=lambda t: t[1], reverse=True))